In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
def featureExtraction(df):
    candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
           'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
           'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
    df = apply_patterns_in_range(df)
    for cp in candle_patterns:
        df[cp] = df[cp].astype(float)

    df = BollingerBandsNonLinear(df)
#     df = ATRFeature(df)
    df = KeltnerChannelsNonLinear(df)
    df = RSINonLinear(df)
    df = MACDNonLinear(df)
    df = BREAKOUT(df)
    df = ichimokuNonLinear(df)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [3]:
BUY = 2
SELL = 0
NONE = 1

In [4]:
def labeling(df):
    df['SIGNAL_LABEL'] = NONE
    df['TP'] = 0
    df['SL'] = 0
    
    for i in range(1,len(df)):
        for j in range(i,len(df)):
            if( df.low[j] <= df.low[i-1] ):
                break
            else:
                if(df.high[j] > 2*(df.close[i] - df.low[i-1] ) + df.close[i]):
                    df.SIGNAL_LABEL[i] = BUY
                    break


        for j in range(i,len(df)):
            if( df.high[j] >= df.high[i-1] ):
                break
            else:
                if(df.low[j] < 2*(df.close[i] - df.high[i-1] ) + df.close[i]):
                    df.SIGNAL_LABEL[i] = SELL
                    break

        
    return df

In [5]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [6]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD

0.0006099999999999994

In [7]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

0.0006000000000000001

In [8]:
SLIT_INDEX = int(0.8*len(df_an))
df_train = df_an.iloc[:SLIT_INDEX]
df_train.reset_index(drop=True, inplace=True)
df_test = df_an.iloc[SLIT_INDEX:]
df_test.reset_index(drop=True, inplace=True)

In [9]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [10]:
df_train = featureExtraction(df_train)
df_train = labeling(df_train)
df_test = featureExtraction(df_test)
df_test = labeling(df_test)


C:\Users\meysam\Desktop\algotramic\technicals\indicators.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_pivot_h'].iloc[i] = last_pivot_h
C:\Users\meysam\Desktop\algotramic\technicals\indicators.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_pivot_l'].iloc[i] = last_pivot_l
C:\Users\meysam\AppData\Local\Temp\ipykernel_11896\1852923117.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.SIGNAL_LABEL[i]

In [11]:
df_train.columns

Index(['time', 'open', 'high', 'low', 'close', 'full_range', 'body_lower',
       'body_upper', 'body_bottom_perc', 'body_top_perc', 'body_perc',
       'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'BB_MA_cross', 'BB_UP_touch',
       'BB_LW_touch', 'EMA_cross', 'KeUp_touch', 'KeLo_touch', 'RSI_level',
       'MACDHIST_level', 'pivots_l', 'pivots_h', 'low_perv', 'high_perv',
       'low_next', 'high_next', 'last_pivot_h', 'last_pivot_l', 'close_perv',
       'BREAK_UP', 'BREAK_DOWN', 'Tenkan_sen', 'Kijun_sen', 'Senkou_Span_A',
       'Senkou_Span_B', 'Tenkan_sen_touch', 'Kijun_sen_touch',
       'Senkou_Span_A_touch', 'Senkou_Span_B_touch', 'Ten_kij_cross',
    

In [12]:
df_train.pivots_h

0        1.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
15913    0.0
15914    1.0
15915    0.0
15916    0.0
15917    0.0
Name: pivots_h, Length: 15918, dtype: float64

In [13]:
len(df_test)

3922

In [14]:

feature_col = [
    'BB_MA_cross', 'BB_UP_touch','BB_LW_touch',
    'EMA_cross', 'KeUp_touch', 'KeLo_touch', 
    'RSI_level',
    'MACDHIST_level',
#      'BREAK_UP', 'BREAK_DOWN',
        'Tenkan_sen_touch', 'Kijun_sen_touch',
       'Senkou_Span_A_touch', 'Senkou_Span_B_touch',
        'Ten_kij_cross','Cloud_cross',
#         'full_range',# this is added ...
#        'body_lower', 'body_upper', 
#         'body_bottom_perc', 'body_top_perc',
#        'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
#     'mid_point', 'mid_point_prev_2', 'body_size_prev',
#        'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'direction',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [15]:
class RNN(nn.Module):
    def __init__(self, batch_size, input_size, hidden_dim,lstm_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.LSTM_layers = lstm_layers
    
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=self.hidden_dim, num_layers=self.LSTM_layers, batch_first=True)
        self.fc = nn.Linear(in_features=self.hidden_dim, out_features=3)
#         self.fc2 = nn.Linear(257, 1)

    def forward(self, x):
        h = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
        c = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
        torch.nn.init.xavier_normal_(h)
        torch.nn.init.xavier_normal_(c)
        out, (hidden, cell) = self.lstm(x, (h,c))
#         out = self.dropout(out)
        out = self.fc(out[:,-1,:]) 
#         out = torch.sigmoid(self.fc2(out))
        return out

In [19]:
class dataset(Dataset):
    def __init__(self,df,sequence_lenght,feature_col):
        self.df = df
        self.sequence_lenght = sequence_lenght
        self.feature_col = feature_col
        self.X = []
        self.Y = [] 
        self.indeces = []
        print("preparing data..")
        self.prepare_data()
        
    def prepare_data(self):
        for i in range(self.sequence_lenght+1,len(self.df)):
#             if self.df.pivots_h[i-1] or self.df.pivots_l[i-1]:
            X_feat = [] 
            for j in range(self.sequence_lenght):
                feature = []
                feature.append(self.df.pivots_h[i-self.sequence_lenght+j])
                feature.append(self.df.pivots_l[i-self.sequence_lenght+j])
                for fc in self.feature_col:
                    feature.append(self.df[fc][i-self.sequence_lenght+j+1])

                X_feat.append(feature)  
            self.X.append(np.array(X_feat))# TODO: use torch tensor 
            self.Y.append(self.df.SIGNAL_LABEL[i])
            self.indeces.append(i)
                
    def get_feature_len(self):
        return self.X[0].shape[1]
    
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self,index):
        return torch.from_numpy(self.X[index]).float(),self.Y[index]

In [20]:
epoch = 1000
batch_size = 600
sequence_lenght = 100

In [21]:
ds_train = dataset(df_train,sequence_lenght,feature_col)
ds_test = dataset(df_test,sequence_lenght,feature_col) 

loader_train = DataLoader(ds_train, batch_size=batch_size)
loader_test = DataLoader(ds_test, batch_size=batch_size)

preparing data..
preparing data..


In [22]:
print(len(loader_train))
print(len(loader_test))
print(len(ds_train))
print(len(ds_test))

27
7
15817
3821


In [23]:
input_size = ds_train.get_feature_len() 
hidden_dim = 60
lstm_layers = 2

In [24]:
model = RNN(batch_size, input_size, hidden_dim,lstm_layers)

In [25]:
weight = torch.Tensor([0.4,0.2,0.4])

In [26]:
criterion = nn.CrossEntropyLoss(weight=weight)

In [27]:
optimizer = optim.Adam(model.parameters(),lr=1e-3)

In [ ]:
model.train()
train_loss = []
test_loss = []
test_accuracy = []
for i in range(epoch):
    loss_epoch_train = 0
    for x_batch, y_batch in loader_train:
        pred = model(x_batch)
        loss = criterion(pred,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_epoch_train += loss.detach().item()
        
    
    loss_epoch_test = 0
    model.eval()
    pred_test = []
    all_label = []
    
    for x_batch, y_batch in loader_test:
        pred = model(x_batch)
        loss = criterion(pred,y_batch)
        loss_epoch_test += loss.detach().item()
        pred_test += list(pred.softmax(dim=1).argmax(dim=1).detach().numpy())
        all_label += list(y_batch.detach().numpy())
        
    train_loss.append(loss_epoch_train/len(loader_train))
    test_loss.append(loss_epoch_test/len(loader_test))
    pred_test = np.array(pred_test)
    all_label = np.array(all_label)
    conf = confusion_matrix(all_label, pred_test)
    print(conf)
    test_accuracy.append((pred_test==all_label).sum()/len(all_label))
    print(train_loss[-1],test_accuracy[-1])

[[ 495  233   11]
 [ 820 1168  376]
 [  97  417  204]]
0.8833796139116641 0.4886155456686731
[[ 483  238   18]
 [ 760 1296  308]
 [ 110  428  180]]
0.8707122184612133 0.512693012300445


In [ ]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    if i > 0:
#         df_test.SIGNAL.loc[idx] = y_perdict[i]
        if df_test.pivots_l[idx-1]==1 and y_perdict[i] ==BUY:
            df_test.SIGNAL.loc[idx] = y_perdict[i]-1
        if df_test.pivots_h[idx-1]==1 and y_perdict[i] ==SELL:
            df_test.SIGNAL.loc[idx] = y_perdict[i]-1

df_test.reset_index(drop=True, inplace=True)

In [ ]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m5 = gt.run_test()

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio =1# abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = 2#abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
print("min balance ",min(df_res_m5['balance']))
cp.show_plot(line_traces=['balance'])